## Class 9 Agenda:
  * **Feature Decomposition and Dimensionality Reduction: PCA**
  * **Regularization: Lasso Regression (L1 regularization for Logistic Regression)**
  * **Regularization: Ridge Regression (L2 regularization for Logistic Regression)**
  * **Regularization: Elastic Net Regression (L1 and L2 combined regularization for Logistic Regression)**

Today we talk about what happens when we have lots of predictor variables (columns/features), and are trying to both generate accurate models, and reduce the number of columns we use to predict/explain what is going on in our dataset.

We may need to do this for a variety of reasons.

In many instances, the number of columns you have in your data can be much larger than the number of samples (examples/rows) you have. One unfortunate result of this can be that we **overfit** our models (many different combinations of the columns in our data can generate the same training results). We need a systematic way to reduce the number of columns we use in our models to just those columns that are the most important for our prediction.

**How can we tell which columns are the most important, and how can we reduce or recombine them in such a way that we just keep/use the most important ones?**

That's the topic of today's lesson.

In general, not all of the columns in our dataset may be critical for predicting a given outcome variable because:

1. Several columns may be highly correlated with each other (provide redundant information) and with the outcome variable.
2. Columns may not be correlated at all with the outcome attribute (have no "predictive ability" in terms of what you're trying to predict).

The two strategies for dealing with both of these cases are called **Dimensionality Reduction** and **Regularization.** Both strategies can be used for supervised classification/regression problems, as well as for unsupervised learning and clustering.

By the end of this lesson you will be able to:

  * use PCA to reduce the size of a given dataset's features to a strictly smaller number of features
  * understand how to interpret the eigenvalues and variance explained by a certain number of PCs after performing PCA
  * use Lasso/L1 regularization for both feature selection and general regression/classification problems
  * use Ridge/L2 regularization for regression and classification problems
  * explain the difference between L1/L2 regularization

In [ ]:
from __future__ import print_function, division, absolute_import  # Python 2/3

In [ ]:
#data handling, model creation/evaluation
import pandas as pd
import numpy as np
import scipy.stats as stats

from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, PolynomialFeatures, StandardScaler
from sklearn.linear_model import Lasso, Ridge, LinearRegression, LogisticRegression, ElasticNet
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
try:
    from sklearn.model_selection import cross_val_score, train_test_split
except ImportError:
    from sklearn.cross_validation import cross_val_score, train_test_split
    
# visualization
%matplotlib inline
import seaborn as sns

We are going to use the [Boston Housing Dataset](http://archive.ics.uci.edu/ml/datasets/Housing) from the UCI repo to explore both Feature Decomposition/Dimensionality Reduction and Regularization.

Here's the dataset schema:

  1. **crime:** per capita crime rate by town
  2. **zone:** proportion of residential land zoned for lots over 25,000 sq.ft.
  3. **industry:** proportion of non-retail business acres per town
  4. **charles:** Charles River dummy variable (1 if tract bounds river; 0 otherwise)
  5. **no:** nitric oxides concentration (parts per 10 million)
  6. **rooms:** average number of rooms per dwelling
  7. **age:** proportion of owner-occupied units built prior to 1940
  8. **distance:** weighted distances to five Boston employment centres
  9. **radial:** index of accessibility to radial highways
  10. **tax:** full-value property-tax rate per 10K dollars
  11. **pupil:** pupil-teacher ratio by town
  12. **aam:** $1000(aam - 0.63)^2$ where aam is the proportion of african americans by town
  13. **lower:** percentage lower income status of the population
  14. **med_price:** Median value of owner-occupied homes in $1000's
  
Let's load the data in:

In [ ]:
housing_columns = ["crime","zone","industry","charles","no","rooms","age",
                   "distance","radial","tax","pupil","aam","lower","med_price"]
housing_features = housing_columns[:-1]
housing_target = housing_columns[-1]
housing_data = pd.read_csv("../data/boston_housing.csv",names=housing_columns)

And compute some basic stats per column:

In [ ]:
housing_data.describe()

And let's look at the correlation matrix:

In [ ]:
sns.heatmap(housing_data.corr())
housing_data.corr()

Let's also plot the histogram of median housing prices across the dataset. This is what we are going to try to predict initially.

In [ ]:
housing_data[housing_target].hist(bins=20)

Let's try and build a linear regression model using this data, using 50-fold cross validation:

In [ ]:
#standardize the features, to get actual interpretable coefficients for each feature
X = StandardScaler().fit_transform(housing_data[housing_features])
y = housing_data[housing_target]

lr = LinearRegression()

#must take absolute value because sklearn makes the errors negative - dont ask me why
mean_squared_errors = np.abs(cross_val_score(lr,X,y,cv=50,scoring='neg_mean_squared_error'))
root_mean_squared_errors = np.sqrt(mean_squared_errors)
print("50-fold mean RMSE: ", np.mean(root_mean_squared_errors))

So, we can expect to be off by ~4,000$ in our prediction if we use this model.

Now let's see what the coefficients of a model trained on all of the data look like:

In [ ]:
coeffs = LinearRegression().fit(X,y).coef_
coeff_df = pd.DataFrame(list(zip(housing_features,np.abs(coeffs))),columns=["features","betas"])
coeff_df.sort_values("betas",ascending=False,inplace=True)
coeff_df

Look at how the last couple features barely impact the linear regression. Let's see if we can't reduce the total number of columns we use without impacting the overall quality of the prediction.

We will try do this using **PCA**.

**PCA**, short for [**Principal Components Analysis**](https://en.wikipedia.org/wiki/Principal_component_analysis) is a method for transforming your initial features into a new feature space such that:
  * All of the new columns are uncorrelated with each other, but are linearly-blended versions of the original columns (features)
  * Each succesive column is less important in terms of how much of the variance (information) in the original data it explains. So, the first column is the "most important" (in a specific statistical sense) column, the 2nd column is "2nd most important" and so on.

We can generate a PCA-based transformation of our features. Let's see that in action:

In [ ]:
pca = PCA()
transformed_pca_x = pca.fit_transform(housing_data[housing_features])
#create component indices
component_names = ["component_"+str(comp) for comp in range(1, len(pca.explained_variance_)+1)]

#generate new component dataframe
transformed_pca_x = pd.DataFrame(transformed_pca_x,columns=component_names)
print(housing_data[housing_features].head())
transformed_pca_x.head()

Once PCA is performed on a given dataset, we get several pieces of information:

  * We get a PCA decomposition of the original features into PCA components, into what is called a component matrix. This matrix tells us how to generate the new components (by blending the original features/columns) and is ordered from most-important to least important
  * We get what are called the **eigenvectors** of each newly-generated component (column), which tells us how important that column is in terms of the amount of the "information" in the original dataset it explains. From the eigenvectors, we can compute the total amount of variance explained per-component as a fraction of the total amount of variance in the original dataset.

In general, **PCA is a useful method for reducing your data to a smaller dimensional representation that you can look at, not necessarily that you can then successfully predict from.**

Let's take a look at both the component matrix and the importance of each PCA dimension.

In [ ]:
#generate component loadings on original features
component_matrix = pd.DataFrame(pca.components_,index=component_names,columns = housing_features)
#add additional columns to describe what
component_matrix["explained_variance_ratio"] = pca.explained_variance_ratio_
component_matrix["eigenvalue"] = pca.explained_variance_
figure = sns.plt.figure(figsize=(18,6))
#add 3 subplots one at a time

#first the component matrix
figure.add_subplot(131)
sns.heatmap(component_matrix.ix[:,:-2])

#then the eigenvalues
figure.add_subplot(132)
sns.plt.plot(range(1,component_matrix.shape[0]+1), component_matrix.eigenvalue)
sns.plt.xlabel("component number")
sns.plt.ylabel("variance explained")

#then the explained variance ratio
figure.add_subplot(133)
sns.plt.plot(range(1,component_matrix.shape[0]+1), component_matrix.explained_variance_ratio)
sns.plt.xlabel("component number")
sns.plt.ylabel("eigenvalue")

In essence, is fundamentally a linear method for projecting the original data onto a smaller-dimensional space (fewer columns) that keeps as much of the variance (distribution) of the original data as possible. What fraction of the original variance is explained by using just the first two PCA components?

In [ ]:
component_matrix.explained_variance_ratio.cumsum()

So it looks like just the first 2 components explain ~97% of the total variance in the dataset. Let's try to use just those 2 components to predict house price and see if it gives us comparable performance:

In [ ]:
mean_squared_errors = np.abs(cross_val_score(lr,transformed_pca_x.ix[:,:2],y,cv=50,scoring='neg_mean_squared_error'))
#apply square root to each fold
root_mean_squared_errors = np.sqrt(mean_squared_errors)
print("50-fold mean RMSE: ", np.mean(root_mean_squared_errors))
print("50-fold std RMSE: ", np.std(root_mean_squared_errors))

So it looks like this gives us pretty terrible performance.

#### Exercise Time

* How many components do we need to keep until we achieve very similar performance?

In [ ]:
pass

Let's visualize all of the components and the binned housing price to see if they help at all in clearing up the relationship between the price and the bulk of the variance (information) in the data:

In [ ]:
transformed_pca_x["housing_price_bin"] = pd.cut(housing_data[housing_target],5)
sns.pairplot(transformed_pca_x,hue="housing_price_bin")

So this doesn't look like it helps very well in this case. Nonetheless, PCA is a useful technique in some cases. 

Specifically, it **can be very very useful when you have hundreds and hundreds of columns and need to drastically reduce the total number of columns you are working with to reduce how long it takes to build your classification model.**

### Exercise Time

* Perform PCA on the `iris` dataset. Attempt to classify the 3 types of irises from each other using just the first 2 components of a PCA-transformed dataset, and using the full untransformed dataset. What is the classification difference?

In [ ]:
iris_data = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/iris/bezdekIris.data",
                        names=["sepal_length","sepal_width","petal_length","petal_width","iris_type"])

le = LabelEncoder()
iris_data.iris_type = le.fit_transform(iris_data.iris_type)
iris_data.head()

Now, let's try something else with the Boston dataset in order to improve the overall predictive accuracy.

Let's generate additional polynomial features up to degree 2 (all pairwise interactions among our original columns), and see if we can't improve the overall RMSE (by adding non-linearities that might be predictive apart from the linear features we've had so far):

In [ ]:
#generate the transformer
pf_2 = PolynomialFeatures(degree=2,interaction_only=True)
#apply it to the data, but ignore the first constant column
pf_2_data = pf_2.fit_transform(housing_data[housing_features])[:,1:]#ignore constant column
print(pf_2_data.shape)
lr = LinearRegression()
#get the mean squared errors per fold, 10 fold cv
absolute_errors = np.abs(cross_val_score(lr,pf_2_data,housing_data[housing_target],cv=10))
#convert each fold's mean squared errors to rmse
rmses = np.sqrt(absolute_errors)
print("10-fold mean RMSE for degree-2 case: ",np.mean(rmses))

So that worked really well, we generated all interaction features, along with our original features, and now we have much lower RMSE! Awesome!

What if we extended this to 3-degree polynomials?

In [ ]:
#generate the polynomial transformer
pf_3 = PolynomialFeatures(degree=3,interaction_only=True)
#apply it to the data, but ignore the first constant column
pf_3_data = pf_3.fit_transform(housing_data[housing_features])[:,1:]#ignore constant column
print(pf_3_data.shape)
lr = LinearRegression()
mean_squared_errors_poly3 = np.abs(cross_val_score(lr,pf_3_data,housing_data[housing_target],cv=10))
rmses_poly3 = np.sqrt(mean_squared_errors_poly3)
print("10-fold mean RMSE for degree-3 case: ",np.mean(rmses_poly3))

Why did this get so much worse?

We have officially horribly overfit our data. Is there anything that can be done in this case?

**YES. We can apply what is called REGULARIZATION!**

Regularization is a method for penalizing overly complicated models, while minimizing out-of-sample (test-set) error. We will talk about 2 kinds of regularization, which have different names, depending on whether they are being used for regression or classification:

 * **Lasso/L1 Regularization**
 * **Ridge/L2 Regularization**

In essence, regularization is a method for adding additional constraints or penalties to a model, with the goal of preventing overfitting and improving generalization.

### Lasso Regression / L1 Regularization for Logistic Regression

In traditional regression/classification, we try to minimize some kind of loss function (we try to make the loss be as close to 0 as possible). In regression, this is usually the **mean squared error** (or root mean squared error). However, with regularization, instead of just minimizing the loss based on the error of the model, we can also add a term that effectively "punishes" more complicated models.

What this means is that as you add more terms (features/columns) to your model, you suffer more added "loss due to complexity" in addition to "loss due to model performance." 

So, how can regularization be used in linear regression to effectively make models simpler?

**You reduce the coefficients for specific columns in your model until they go to zero (or very close to zero) for your model, which has the net effect of not using those features (or using them very sparingly)**

More mathematically, Lasso/L1 regularization adds an absolute-value penalty to the loss function:

$$loss_{lasso} = |Xw-Y|^2 + \lambda|w|$$

Whereas the non-regularized version is simply:

$$loss = |Xw-Y|^2$$

So, L1 regularization adds a penalty (based on $\lambda$) to the regular loss function. Since each non-zero coefficient adds to the penalty, it forces weak features to have zero as coefficients. Thus L1 regularization produces sparse solutions, inherently performing feature selection.

However, because this is an absolute value-based regularization (and absolute values are no-fun when it comes to taking derivatives), L1/Lasso regularization can lead to non-unique solutions (especially when you have highly correlated features...why?).

For regression, Scikit-learn offers the `Lasso` and `LassoCV` functions, and for Logistic regression it offers the `l1` penalty parameter for classification. Let's use the `Lasso` with full regularization (when $\lambda=0$) on the degree-3 case:

In [ ]:
lasso = Lasso(max_iter=2000)#increase the number of iterations for lasso to prevent lack of convergence
mean_squared_errors_poly3_lasso = np.abs(cross_val_score(lasso,pf_3_data,housing_data[housing_target],cv=10))
rmses_lasso_poly3 = np.sqrt(mean_squared_errors_poly3_lasso)
print("10-fold mean RMSE for degree-3 case, strongest lasso regularization: ",np.mean(rmses_lasso_poly3))

Wow! That worked very well! We now have a much better model! Let's examine how many of the coefficients are non-zero when we fit on the full dataset:

In [ ]:
lasso_all = Lasso().fit(pf_3_data,housing_data[housing_target])
print("Number of features in the model: ",len(lasso_all.coef_))
non_zero_features_mask = np.abs(lasso_all.coef_)>0.0001
print(non_zero_features_mask)
print("Number of non-zero features in the model: ",np.sum(non_zero_features_mask.astype(int)))
print("Fraction of total features used: ",float(np.sum(non_zero_features_mask.astype(int)))/len(lasso_all.coef_))

So, with full lasso regularization, we get a model that uses less than 40% of the original columns, and performs better than the model that only uses the original features or the combination of the original features and the pairwise interactions!

#### Exercise Time!

 * Try the lasso method but generating the 4th order polynomial features (use `PolynomialFeatures(degree=4,interactions_only=True)` and the same `Lasso()` and `cross_val_score` functions). Does our RMSE improve yet again?
 * Try the lasso method for the poly3 case, but change the $\lambda$ (called $\alpha$ in sklearn) parameter to `0.2`; What happens to the RMSE? What happens to the fraction of non-zero features in the model trained on all the data?

In [ ]:
pass

### Ridge Regression / L2 Regularization for Logistic Regression

So, Lasso looks really excellent to use a regularization method. However, as I'd mentioned before, it is difficult to use across the board for all regression/classification problems, because it can create non-unique solutions, or solutions don't always converge (because of the problem with taking derivatives of a function that is not differentiable everywhere).

So, what can be done? We can create a penalty term that is smooth and therefore differentiable everywhere. Specifically, we can use what is called **Ridge/L2 regularization.** Here, the loss is slightly different:

$$loss_{ridge} = |Xw-Y|^2 + \lambda|w|^2$$

The act of squaring the weight features makes the penalty differentiable (because the values now smoothly transition to 0). So, squaring the L1 penalty turns it into an L2 penalty Square the Lasso and get a Ridge (or something).

Since the coefficients are squared in the penalty expression, it has a different effect from L1-norm, namely it forces the coefficient values to be spread out more equally. For correlated features, it means that they tend to get similar coefficients (whereas in L1 regularization, one of the terms will be forced to 0).

**The effect of this is that models are much more stable (coefficients do not fluctuate on small data changes as is the case with unregularized or L1 models). So while Ridge/L2 regularization does not perform feature selection the same way as L1 does, it is much more useful for feature *interpretation*; a predictive feature will get a non-zero coefficient, which is often not the case with L1.**

Let's use the full-penalty Ridge regression and see what happens:

In [ ]:
ridge = Ridge()
ridge_cv_squared_errors = np.abs(cross_val_score(ridge,pf_3_data,housing_data[housing_target],cv=10))
rmse_ridge_cv = np.sqrt(ridge_cv_squared_errors)
print("10-fold mean RMSE for degree-3 case, strongest lasso regularization: ",np.mean(rmse_ridge_cv))

#### Exercise Time

* Try to use both lasso and ridge on the degree-2 polynomial version of the data, do either of them reduce the RMSE? What does our result tell us about the full degree-2 model?

### Lasso vs. Ridge

#### Lasso:
* produces sparse models 
* is useful for strong feature selection in order to improve model performance, or to minimize the number of explanatory variables.
* can produce non-unique solutions (when some features are very strongly correlated)
* can produce very different solutions depending on slight changes in features (because of non-uniqueness)

#### Ridge:
* produces stable models with smooth non-zero coefficients across features. 
* is useful for data interpretation, understanding what features, even when correlated, may be used in combination to predict the response.
* may tell you something about how the data itself was generated.

You can combine both Lasso and Ridge models into a single penalized model (that uses a weighted combination of Lasso and Ridge regression). This is called the `ElasticNet`.

### General Strategy and Takeaways

* PCA is more useful for explanation/data exploration than it is for classification/regression when you have few (< 100) features.
* When generating polynomial features, generate the lowest-order polynomial interactions first, drop the constant term (the first column in the matrix), and try to use the full model for regression/classification.
* If you are trying for data explanation, use Ridge/L2 regularization
* If you are going for sparse models, use Lasso/L1, but realize that the solution may be non-unique. In general, Lasso doesn't always perform better than Ridge.
* `ElasticNet` requires significantly more tuning than either method, but can lead to the highest performing "linear" models. **As always, with great power comes great responsibility, so use all of these methods wisely.**